In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
NTHREAD = cpu_count()

import warnings
warnings.filterwarnings("ignore")

from IPython.display import display
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 50)

In [2]:
train  = pd.read_csv('../input/application_train.csv.zip')
train['is_train'] = 1

test  = pd.read_csv('../input/application_test.csv.zip')
test['is_train'] = 0

trte = pd.concat([train, test], ignore_index=True)

In [3]:
prev = pd.read_csv('../input/previous_application.csv.zip')
prev.sort_values('DAYS_DECISION', inplace=True)

In [4]:
user_id = pd.read_csv('../data/user_id_v8.csv.gz')

In [5]:
dup = user_id[user_id.duplicated('user_id', False)]
cnt = dup.groupby('user_id').size()
cnt.name = 'dup_cnt'
cnt = cnt.reset_index()
cnt['dup_id'] = range(cnt.shape[0])

dup = pd.merge(dup, cnt, on='user_id', how='left')
dup = pd.merge(dup, trte, on='SK_ID_CURR', how='left')
dup.sort_values(['user_id', 'DAYS_BIRTH'], ascending=[True, False], inplace=True)

dup['seq'] = 1
dup['seq'] = dup.groupby('user_id').seq.cumsum()-1

col = train.columns.tolist() + [c for c in dup.columns if c not in train.columns]
for c in ['dup_id', 'dup_cnt', 'user_id']:
    col.remove(c)
col = ['user_id'] + col

dup = dup[col].reset_index(drop=True)
dup

,user_id,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,...,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,is_train,seq
0,18.0,100279,0.0,Cash loans,M,N,Y,0,180000.0,254700.0,16407.0,225000.0,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.072508,-23946,365243,-6935.0,-3948,NaN,1,0,0,1,0,0,NaN,1.0,1,1,WEDNESDAY,16,0,0,0,0,0,0,XNA,NaN,0.458685,0.528093,0.3041,0.1854,0.9791,0.7144,...,0.7182,0.0000,0.32,0.2759,0.3333,0.0417,0.0000,0.2454,0.2861,0.0311,0.0092,reg oper account,block of flats,0.2230,Panel,No,0.0,0.0,0.0,0.0,-975.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,5.0,1,0
1,18.0,100124,0.0,Cash loans,M,N,Y,0,225000.0,448272.0,28647.0,396000.0,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,0.072508,-24051,365243,-7040.0,-4053,NaN,1,0,0,1,1,1,NaN,1.0,1,1,WEDNESDAY,17,0,0,0,0,0,0,XNA,NaN,0.710318,0.554947,0.2959,0.1785,0.9786,0.7076,...,0.7115,0.0000,0.32,0.2759,0.3333,0.3750,0.0000,0.2454,0.2741,0.0000,0.0062,reg oper account,block of flats,0.2131,Panel,No,0.0,0.0,0.0,0.0,-1080.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,8.0,1,1
2,38.0,100955,0.0,Cash loans,F,N,Y,0,112500.0,101880.0,10053.0,90000.0,Unaccompanied,Pensioner,Lower secondary,Single / not married,House / apartment,0.030755,-22321,365243,-13611.0,-2523,NaN,1,0,0,1,0,0,NaN,1.0,2,2,FRIDAY,10,0,0,0,0,0,0,XNA,NaN,0.664900,0.504681,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,2.0,1.0,-1012.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,3.0,1,0
3,38.0,190916,NaN,Cash loans,F,N,Y,0,135000.0,238500.0,24565.5,238500.0,Unaccompanied,Pensioner,Lower secondary,Single / not married,House / apartment,0.030755,-22609,365243,-73.0,-2811,NaN,1,0,0,1,1,0,NaN,1.0,2,2,SATURDAY,10,0,0,0,0,0,0,XNA,0.279026,0.504692,0.466864,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,-1300.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0,0,1
4,40.0,178701,0.0,Revolving loans,F,N,Y,0,90000.0,270000.0,13500.0,270000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-16416,-1223,-2542.0,-4382,NaN,1,1,0,1,0,0,NaN,2.0,2,2,WEDNESDAY,12,0,0,0,0,1,1,Business Entity Type 3,0.479804,0.514119,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

# Forwards

In [6]:
dup[dup.SK_ID_CURR==100041]

,user_id,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,...,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,is_train,seq
1132,4402.0,100041,0.0,Cash loans,F,N,N,0,112500.0,450000.0,44509.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.008575,-12158,-1275,-6265.0,-2009,NaN,1,1,1,1,1,0,Sales staff,2.0,2,2,MONDAY,19,0,0,0,0,0,0,Self-employed,NaN,0.663158,0.678568,0.0948,0.0792,0.9861,0.8096,...,0.8121,NaN,0.0,0.1724,0.1667,NaN,0.0903,NaN,0.0592,NaN,NaN,reg oper account,block of flats,0.0749,"Stone, brick",No,0.0,0.0,0.0,0.0,-1285.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [7]:
dup[dup.SK_ID_CURR==181102]

,user_id,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,...,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,is_train,seq
1133,4402.0,181102,0.0,Cash loans,F,Y,N,0,67500.0,302076.0,23508.0,270000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.008575,-12395,-1146,-6502.0,-2246,14.0,1,1,0,1,1,1,Sales staff,2.0,2,2,TUESDAY,16,0,0,0,0,0,0,Self-employed,NaN,0.664866,0.551381,0.0948,0.0792,0.9856,0.8028,...,0.8054,NaN,0.0,0.1724,0.1667,NaN,0.0903,NaN,0.0592,NaN,NaN,reg oper account,block of flats,0.0749,"Stone, brick",No,2.0,0.0,2.0,0.0,-1522.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,5.0,1,1


In [8]:
prev[prev.SK_ID_CURR==100041]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1133178,1786922,100041,Consumer loans,3268.485,15516.0,16285.5,0.0,15516.0,THURSDAY,12,Y,1,0.0,NaN,NaN,XAP,Approved,-2701,Cash through the bank,XAP,Family,New,Mobile,POS,XNA,Country-wide,156,Connectivity,6.0,high,POS mobile with interest,365243.0,-2670.0,-2520.0,-2520.0,-2509.0,1.0
1546899,2305060,100041,Consumer loans,7297.515,58486.5,64278.0,0.0,58486.5,SUNDAY,14,Y,1,0.0,NaN,NaN,XAP,Approved,-1508,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,4000,Consumer electronics,12.0,high,POS household with interest,365243.0,-1477.0,-1147.0,-1147.0,-1095.0,0.0


In [9]:
prev[prev.SK_ID_CURR==181102]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
800131,1561592,181102,Consumer loans,7297.515,58486.5,64278.0,0.0,58486.5,SUNDAY,14,Y,1,0.0,NaN,NaN,XAP,Approved,-1745,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,4000,Consumer electronics,12.0,high,POS household with interest,365243.0,-1714.0,-1384.0,-1384.0,-1332.0,0.0
124560,1272122,181102,Cash loans,44509.500,450000.0,450000.0,NaN,450000.0,MONDAY,19,Y,1,NaN,NaN,NaN,XNA,Approved,-237,XNA,XAP,Unaccompanied,Refreshed,XNA,Cash,x-sell,Contact center,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-207.0,123.0,365243.0,365243.0,0.0
1206352,1524441,181102,Revolving loans,NaN,0.0,0.0,NaN,NaN,WEDNESDAY,14,Y,1,NaN,NaN,NaN,XAP,Canceled,-153,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Card Street,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
col_num = prev.select_dtypes(exclude='O').columns.tolist()
col_cat = prev.select_dtypes('O').columns.tolist()

In [11]:
date_feature = ['DAYS_DECISION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
               'DAYS_LAST_DUE', 'DAYS_TERMINATION']

keys = ['NAME_CONTRACT_TYPE', 'AMT_ANNUITY', 'AMT_APPLICATION', 'AMT_CREDIT', 
        'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE', 'WEEKDAY_APPR_PROCESS_START', 
        'HOUR_APPR_PROCESS_START', 'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
        'RATE_DOWN_PAYMENT', 'NAME_CONTRACT_STATUS', 'NAME_TYPE_SUITE', 'PRODUCT_COMBINATION']

In [12]:
for c in date_feature:
    col_num.remove(c)

In [13]:
prev[col_num] = prev[col_num].fillna(-1) # awful 
prev[col_cat] = prev[col_cat].fillna('-NA-')

In [14]:
prev[prev.SK_ID_CURR==181102]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
800131,1561592,181102,Consumer loans,7297.515,58486.5,64278.0,0.0,58486.5,SUNDAY,14,Y,1,0.0,-1.0,-1.0,XAP,Approved,-1745,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,4000,Consumer electronics,12.0,high,POS household with interest,365243.0,-1714.0,-1384.0,-1384.0,-1332.0,0.0
124560,1272122,181102,Cash loans,44509.500,450000.0,450000.0,-1.0,450000.0,MONDAY,19,Y,1,-1.0,-1.0,-1.0,XNA,Approved,-237,XNA,XAP,Unaccompanied,Refreshed,XNA,Cash,x-sell,Contact center,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-207.0,123.0,365243.0,365243.0,0.0
1206352,1524441,181102,Revolving loans,-1.000,0.0,0.0,-1.0,-1.0,WEDNESDAY,14,Y,1,-1.0,-1.0,-1.0,XAP,Canceled,-153,XNA,XAP,-NA-,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,-1.0,XNA,Card Street,NaN,NaN,NaN,NaN,NaN,-1.0


## Clean Up prev

In [15]:
def forwards(args):
    start_ix, last_ix = args
    li = []
    curr_id_bk, seq_bk, day_bk = None, None, None
    for i in range(start_ix, last_ix):
        curr_id, seq, day = dup.iloc[i, 1], dup.iloc[i, -1], dup.iloc[i, 18] 
        if seq ==0:
            curr_id_bk, seq_bk, day_bk = curr_id, seq, day
            continue
        else:
            day_diff = day - day_bk
            
            if day_diff==0:
                curr_id_bk, seq_bk, day_bk = curr_id, seq, day
                continue
            
            prev_bk = prev[prev.SK_ID_CURR==curr_id_bk]
            prev_bk['SK_ID_CURR'] = curr_id
            for c in date_feature:
                prev_bk.loc[prev[c]!=365243, c] += day_diff
            
            prev_cur = prev[prev.SK_ID_CURR==curr_id]
            
            # ==== augmentation ====
            a = prev_bk.groupby(keys).size()
            a.name = 'a'
            b = prev_cur.groupby(keys).size()
            b.name = 'b'
            
            tmp = pd.concat([a, b], axis=1, join='outer')
            tmp[['a', 'b']] = tmp[['a', 'b']].fillna(0)
            tmp['need'] = tmp['a'] - tmp['b']
            
            base = tmp[tmp.need>0][['need']]
            
            aug = pd.DataFrame()
            while base.shape[0] > 0:
                aug = aug.append(base.drop('need', axis=1))
                
                base.need -= 1
                base = base[base.need>0]
            
            if aug.shape[0]==0:
                df = pd.DataFrame()
            else:
                df = pd.merge(aug.reset_index(), prev_bk, on=keys, how='left' )
                df = pd.concat([df, prev_cur])
            # ==== augmentation end ====
            
            
            li.append(df)
        
        curr_id_bk, seq_bk, day_bk = curr_id, seq, day
        
    return li

In [16]:
pd.concat(forwards((0, 2)))

""


In [17]:
argmin = dup.groupby('user_id').seq.apply(np.argmin)
argmax = dup.groupby('user_id').seq.apply(np.argmax)
argmax +=1

argss = list(zip(argmin.values, argmax.values ))

In [18]:
pool = Pool(NTHREAD)
callback = pool.map(forwards, argss)
pool.close()

In [19]:
callback2 = sum( callback , [])

In [20]:
prev_cleaned = pd.concat(callback2)

In [21]:
prev_cleaned_id = prev_cleaned.SK_ID_CURR.tolist()

In [22]:
prev_ = prev[~prev.SK_ID_CURR.isin(prev_cleaned_id)]

In [23]:
prev_new = pd.concat([prev_cleaned, prev_], ignore_index=True)

## check

In [24]:
prev.SK_ID_CURR.unique().shape, prev_new.SK_ID_CURR.unique().shape

((338857,), (338857,))

In [25]:
prev.SK_ID_PREV.unique().shape, prev_new.SK_ID_PREV.unique().shape

((1670214,), (1670214,))

In [26]:
prev[~prev.SK_ID_PREV.isin(prev_new.SK_ID_PREV)]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL


In [27]:
prev_new[prev_new.SK_ID_CURR==181102]

,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,CHANNEL_TYPE,CNT_PAYMENT,CODE_REJECT_REASON,DAYS_DECISION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_TERMINATION,FLAG_LAST_APPL_PER_CONTRACT,HOUR_APPR_PROCESS_START,NAME_CASH_LOAN_PURPOSE,NAME_CLIENT_TYPE,NAME_CONTRACT_STATUS,NAME_CONTRACT_TYPE,NAME_GOODS_CATEGORY,NAME_PAYMENT_TYPE,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,NAME_SELLER_INDUSTRY,NAME_TYPE_SUITE,NAME_YIELD_GROUP,NFLAG_INSURED_ON_APPROVAL,NFLAG_LAST_APPL_IN_DAY,PRODUCT_COMBINATION,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,SELLERPLACE_AREA,SK_ID_CURR,SK_ID_PREV,WEEKDAY_APPR_PROCESS_START
1696,3268.485,15516.0,16285.5,0.0,15516.0,Country-wide,6.0,XAP,-2938,365243.0,-2907.0,-2757.0,-2757.0,-2746.0,Y,12,XAP,New,Approved,Consumer loans,Mobile,Cash through the bank,POS,XNA,Connectivity,Family,high,1.0,1,POS mobile with interest,0.0,-1.0,-1.0,156,181102,1786922,THURSDAY
1697,7297.515,58486.5,64278.0,0.0,58486.5,Country-wide,12.0,XAP,-1745,365243.0,-1714.0,-1384.0,-1384.0,-1332.0,Y,14,XAP,Refreshed,Approved,Consumer loans,Consumer Electronics,Cash through the bank,POS,XNA,Consumer electronics,Family,high,0.0,1,POS household with interest,0.0,-1.0,-1.0,4000,181102,1561592,SUNDAY
1698,44509.500,450000.0,450000.0,-1.0,450000.0,Contact center,12.0,XAP,-237,365243.0,-207.0,365243.0,123.0,365243.0,Y,19,XNA,Refreshed,Approved,Cash loans,XNA,XNA,Cash,x-sell,XNA,Unaccompanied,middle,0.0,1,Cash X-Sell: middle,-1.0,-1.0,-1.0,-1,181102,1272122,MONDAY
1699,-1.000,0.0,0.0,-1.0,-1.0,Credit and cash offices,-1.0,XAP,-153,NaN,NaN,NaN,NaN,NaN,Y,14,XAP,Repeater,Canceled,Revolving loans,XNA,XNA,XNA,XNA,XNA,-NA-,XNA,-1.0,1,Card Street,-1.0,-1.0,-1.0,-1,181102,1524441,WEDNESDAY


# merge with clean prev

In [28]:
prev = pd.read_csv('../input/previous_application.csv.zip')
col = prev.columns.tolist()
prev.drop(['SK_ID_CURR']+date_feature, axis=1, inplace=True)

In [29]:
prev_new_base = prev_new[['SK_ID_PREV', 'SK_ID_CURR']+date_feature]

In [30]:
prev_new_base.shape

(1671673, 8)

In [31]:
prev.shape

(1670214, 30)

In [32]:
prev_new2 = pd.merge(prev_new_base, prev, on='SK_ID_PREV', how='left')[col]

In [33]:
prev_new2

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2821489,340385,Cash loans,9109.485,90000.0,95940.0,NaN,90000.0,THURSDAY,14,Y,1,NaN,NaN,NaN,XNA,Approved,-4821,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-4791.0,-4461.0,365243.0,365243.0,1.0
1,1940786,340385,Consumer loans,7969.320,60525.0,42367.5,18157.5,60525.0,THURSDAY,11,Y,1,0.326727,NaN,NaN,XAP,Approved,-4996,Cash through the bank,XAP,NaN,New,Furniture,POS,XNA,Stone,30,Furniture,6.0,middle,POS industry with interest,365243.0,-4952.0,-4802.0,-4802.0,-4797.0,0.0
2,2559483,340385,Cash loans,12185.370,229500.0,266760.0,NaN,229500.0,WEDNESDAY,14,Y,1,NaN,NaN,NaN,XNA,Approved,-1252,Cash through the bank,XAP,Other_A,Refreshed,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,36.0,middle,Cash X-Sell: middle,365243.0,-1222.0,-172.0,-1102.0,-1094.0,1.0
3,2078085,340385,Cash loans,14312.880,270000.0,313839.0,NaN,270000.0,TUESDAY,14,Y,1,NaN,NaN,NaN,XNA,Approved,-644,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,36.0,middle,Cash X-Sell: middle,365243.0,-614.0,436.0,365243.0,365243.0,1.0
4,2553803,340385,Revolving loans,2250.000,45000.0,45000.0,NaN,45000.0,THURSDAY,14,Y,1,NaN,NaN,NaN,XAP,Approved,-432,XNA,XAP,Unaccompanied,Repeater,XNA,Cards,walk-in,Credit and cash offices,-1,XNA,0.0,XNA,Card Street,365243.0,365243.0,365243.0,365243.0,365243.0,0.0
5,1203723,340385,Cash loans,NaN,0.0,0.0,NaN,NaN,THURSDAY,14,Y,1,NaN,NaN,NaN,XNA,Canceled,-432,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
6,2492530,340385,Cash loans,4266.225,99000.0,125136.0,NaN,99000.0,THURSDAY,14,Y,1,NaN,NaN,NaN,XNA,Approved,-432,Cash through the bank,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,48.0,low_normal,Cash X-Sell: low,365243.0,-402.0,1008.0,365243.0,365243.0,1.0
7,2071598,340385,Cash loans,NaN,0.0,0.0,NaN,NaN,THURSDAY,14,Y,1,NaN,NaN,NaN,XNA,Canceled,-432,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
8,2358616,340385,Cash loans,NaN,0.0,0.0,NaN,NaN,WEDNESDAY,12,Y,1,NaN,NaN,NaN,XNA,Refused,-412,XNA,HC,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN
9,2787081,340385,Cash loans,NaN,0.0,0.0,NaN,NaN,FRIDAY,13,Y,1,NaN,NaN,NaN,XNA,Refused,-410,XNA,HC,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Cash,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
prev_new2[prev_new2.SK_ID_CURR==181102]

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
1696,1786922,181102,Consumer loans,3268.485,15516.0,16285.5,0.0,15516.0,THURSDAY,12,Y,1,0.0,NaN,NaN,XAP,Approved,-2938,Cash through the bank,XAP,Family,New,Mobile,POS,XNA,Country-wide,156,Connectivity,6.0,high,POS mobile with interest,365243.0,-2907.0,-2757.0,-2757.0,-2746.0,1.0
1697,1561592,181102,Consumer loans,7297.515,58486.5,64278.0,0.0,58486.5,SUNDAY,14,Y,1,0.0,NaN,NaN,XAP,Approved,-1745,Cash through the bank,XAP,Family,Refreshed,Consumer Electronics,POS,XNA,Country-wide,4000,Consumer electronics,12.0,high,POS household with interest,365243.0,-1714.0,-1384.0,-1384.0,-1332.0,0.0
1698,1272122,181102,Cash loans,44509.500,450000.0,450000.0,NaN,450000.0,MONDAY,19,Y,1,NaN,NaN,NaN,XNA,Approved,-237,XNA,XAP,Unaccompanied,Refreshed,XNA,Cash,x-sell,Contact center,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-207.0,123.0,365243.0,365243.0,0.0
1699,1524441,181102,Revolving loans,NaN,0.0,0.0,NaN,NaN,WEDNESDAY,14,Y,1,NaN,NaN,NaN,XAP,Canceled,-153,XNA,XAP,NaN,Repeater,XNA,XNA,XNA,Credit and cash offices,-1,XNA,NaN,XNA,Card Street,NaN,NaN,NaN,NaN,NaN,NaN


# output

In [35]:
prev_new2.to_csv('../data/prev_forwards_v3.csv.gz', index=False, compression='gzip')